# Project 1 - Team BAK

## Step 1 - Getting started

In [26]:
#Import some libraries
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import datetime
from helpfulfun import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
# import the datasets
train_list = np.genfromtxt("train.csv", dtype=None, delimiter=",", skip_header =1, unpack=True, encoding=None)
train = np.array(train_list)

test_list = np.genfromtxt("test.csv", dtype=None, delimiter=",", skip_header =1, unpack=True, encoding=None)
test = np.array(test_list)

In [28]:
x_te = test[2:].T

In [29]:
y = train[1]
x_tr = train[2:].T
print("x: ", x_tr.shape, " y: ", y.shape)

x:  (250000, 30)  y:  (250000,)


## Step 2 - Preprocessing

In [30]:
y_tr = np.where(y == "s",1,0)
print(y_tr.shape, " and y[:5]: ", y_tr[:5])

(250000,)  and y[:5]:  [1 0 0 0 0]


In [31]:
# delete features with more than 30% NaN values
x_tr_prep = np.delete(x_tr, 0, 1) # infer this one later
for i in [4, 4, 4, 9, 19, 19, 19, 19, 19, 19]:
    x_tr_prep = np.delete(x_tr_prep, i, 1)
x_tr_prep.shape

(250000, 19)

## Step 3 - Implement ML Methods

#### Linear regression using gradient descent

In [32]:
def mean_squared_error_gd(y, tx, initial_w, max_iters, gamma):
    """The Gradient Descent (GD) algorithm.
        
    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,2)
        initial_w: numpy array of shape=(2, ). The initial guess (or the initialization) for the model parameters
        max_iters: a scalar denoting the total number of iterations of GD
        gamma: a scalar denoting the stepsize
        
    Returns:
        losses: a list of length max_iters containing the loss value (scalar) for each iteration of GD
        ws: a list of length max_iters containing the model parameters as numpy arrays of shape (2, ), for each iteration of GD 
    """
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        gradient = compute_gradient(y, tx, w)
        loss = compute_loss(y, tx, w)
        
        w = w - gamma * gradient
        
        # store w and loss
        ws.append(w)
        losses.append(loss)
        print("GD iter. {bi}/{ti}: loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))

    return losses, ws
    

In [33]:
mean_squared_error_gd(y_tr, tx=x_tr_prep, initial_w=np.zeros((x_tr_prep.shape[1], )), max_iters=50, gamma=.7)

ValueError: data type must provide an itemsize

#### Linear regression using stochastic gradient descent


In [5]:
def mean_squared_error_sgd(y, tx, initial_w, max_iters, gamma):
    """The Stochastic Gradient Descent algorithm (SGD).
            
    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,2)
        initial_w: numpy array of shape=(2, ). The initial guess (or the initialization) for the model parameters
        batch_size: a scalar denoting the number of data points in a mini-batch used for computing the stochastic gradient
        max_iters: a scalar denoting the total number of iterations of SGD
        gamma: a scalar denoting the stepsize
        
    Returns:
        losses: a list of length max_iters containing the loss value (scalar) for each iteration of SGD
        ws: a list of length max_iters containing the model parameters as numpy arrays of shape (2, ), for each iteration of SGD 
    """
    
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    
    for n_iter in range(max_iters):
        for minibatch_y, minibatch_tx in batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
            gradient = compute_stoch_gradient(minibatch_y, minibatch_tx, w)
            loss = compute_loss(y, tx, w)
            w = w - gamma * gradient
        # store w and loss
            ws.append(w)
            losses.append(loss)

        print("SGD iter. {bi}/{ti}: loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    return losses, ws
    

#### Least squares regression using normal equations

In [6]:
def least_squares(y, tx):
    opt_weights = np.linalg.solve(tx.T.dot(tx)).dot(tx.T.dot(y))
    e = y - tx.dot(opt_weights)
    mse = 1/(2*len(y)) * e.T.dot(e)
    return opt_weights, mse

#### Ridge regression using normal equations

In [7]:
def ridge_regression(y, tx, lambda_ ):
    aI = 2 * tx.shape[0] * lambda_ * np.identity(tx.shape[1])
    a = tx.T.dot(tx) + aI
    b = tx.T.dot(y)
    return np.linalg.solve(a, b)

#### Logistic regression using gradient descent or SGD (y ∈ {0, 1})

In [38]:
# calculating loss with sigmoid and using gradient descent
def logistic_regression(y, tx, w):
    loss = calculate_loss_lr(y, tx, w)
    gradient = calculate_gradient_lr(y, tx, w)
    hessian = calculate_hessian(y, tx, w)
    return loss, gradient, hessian

In [39]:
def logistic_regression_demo(y, x):
    # init parameters
    max_iter = 50
    threshold = 1e-5
    lambda_ = 0.1
    gamma = 1.
    losses = []

    # build tx
    tx = np.c_[np.ones((y.shape[0], 1)), x]
    w = np.zeros((tx.shape[1], 1))

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and gradient descent on w.
        loss, gradient, hessian = logistic_regression(y, tx, w)
        w -= gamma * np.linalg.solve(hessian, gradient)
        
        print("Current iteration={i}, the loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break


In [ ]:
logistic_regression_demo(y, x)